# Indexing

In this hands-on exercise we will work with indexing data in BigTable, which is very similar to the open source project called HBase. 

We will all write to the same BigTable instance, so in order to have different tablenames we use the studentID, so please start by changing that to something else!

In [1]:
# Should be changed!
studentID="00"

In [2]:
from google.cloud import bigtable
from google.cloud.bigtable import column_family
from google.cloud.bigtable import row_filters
import datetime
import json

First, we need to set up the client for accessing BigTable

In [3]:
project_id="big-data-course-235920"
instance_id="big-data-course-bigtable"
table_id="songstable"+studentID

client = bigtable.Client(project=project_id, admin=True).instance(instance_id)

Then we will use the client to create a table. In order for the table to hold any data, at least one column family needs to be created. We will create a column family named "cf". There is a feature in BigTable and HBase, which means that when a row is overwritten the previous value is stored and can be retrieved. When a column family is created it must be configured how many versions of the row is retained. We will not use this feature and therefore set the number of versions to store to just one. 

In [4]:
table = client.table(table_id)
column_family_id="cf"
column_families = {column_family_id: column_family.MaxVersionsGCRule(1)}
if not table.exists():
    table.create(column_families=column_families)
else:
    print("Table {} already exists.".format(table_id))
    table.truncate(timeout=200)

Table songstable00 already exists.


Once the table is created it can be seen in the web-client.

Now, we will put some data in the table. A sample row from the One Million Songs dataset looks like this:

| Artist | Song              | Year | Duration  | Loudness | Key | Tempo   |
|--------|-------------------|------|-----------|----------|-----|---------|
| Queen  | Bohemian Rhapsody | 1975 | 354.2722  | -10.872  | 10  | 71.568  |
| Queen  | Under Pressure    | 1986 | 227.36934 | -5.906   | 2   | 123.966 |

If we want to put these in a table, so that we can look up any artist and get the songs of that artist in order of the release year. What would the rowkey look like for this?

If we just want to put two rows into the table it can be done like this:

In [5]:
songs=spark.sparkContext.parallelize([{"artist": "Queen", "year": "1974", "track_name": "Bohemian Rhapsody"}, 
                   {"artist": "Queen", "year": "1986", "track_name": "Under Pressure"}])

def create_row(song):
    client = bigtable.Client(project=project_id, admin=True).instance(instance_id)
    table = client.table(table_id)
    row_key=song["track_name"].encode()
    row=table.row(row_key)
    row.set_cell(column_family_id,
                "song".encode(),
                json.dumps(song).encode())
    table.mutate_rows([row])
    return True

songs_indexed=songs.map(lambda song: create_row(song)).count()
songs_indexed

2

### Retrieving one row

We can get one row from the table if we know the rowkey. 

In [6]:
key = "Bohemian Rhapsody".encode()

row = table.read_row(key)
cell = row.cells[column_family_id]["song".encode()][0]
print(cell.value.decode('utf-8'))

{"artist": "Queen", "year": "1974", "track_name": "Bohemian Rhapsody"}


### Scanning all the rows

Since we only have to rows in the table, it is possible to scan the full table. This can be done with the 

In [7]:
partial_rows = table.read_rows()

for row in partial_rows:
    cell = row.cells[column_family_id]["song".encode()][0]
    print(cell.value.decode('utf-8'))


{"artist": "Queen", "year": "1974", "track_name": "Bohemian Rhapsody"}
{"artist": "Queen", "year": "1986", "track_name": "Under Pressure"}


### Scanning from one rowkey to another

We can specify where to start scanning and where to end. Below we scan from "A" to "Z" and therefore we get both rows. Try to change the start_key and end_key and see how it affects the results.

In [8]:
scanned_rows = table.read_rows(start_key="A".encode(), end_key="Z".encode())

for row in scanned_rows:
    cell = row.cells[column_family_id]["song".encode()][0]
    print(cell.value.decode('utf-8'))

{"artist": "Queen", "year": "1974", "track_name": "Bohemian Rhapsody"}
{"artist": "Queen", "year": "1986", "track_name": "Under Pressure"}


We can delete all rows from the table with the truncate() function.

In [9]:
table.truncate(timeout=200)

### Indexing more songs

We are now ready to index more songs. We will work with the songs of artists that start with "Queen". There should be 457 of those.

Our task is to load data into the the table, such that we can retrieve all the songs by "Queen", "Queens Club", "Queens Of The Stone Age" etc. ordered by release year. For some songs the year is zero. Don't worry about those.

In [10]:
all_queen_songs=spark.read.csv("gs://big-data-course-datasets/one-million-songs/one-million-songs.csv", header=True).rdd \
  .filter(lambda x: x["artist_name"].startswith("Queen"))

In [11]:
all_queen_songs.take(10)

[Row(artist_name='Queen + Paul Rodgers', track_name='I Want To Break Free (Live In Ukraine)', year='0', duration='235.25832', loudness='-8.141', key='2', tempo='114.133'),
 Row(artist_name='Queen Latifah', track_name="Name Callin' (Explicit Soundtrack LP Version)", year='0', duration='230.66077', loudness='-7.852', key='0', tempo='86.097'),
 Row(artist_name='Queen Omega', track_name='Brighter Day', year='0', duration='200.22812', loudness='-9.216', key='7', tempo='133.917'),
 Row(artist_name='Queen', track_name='Love Of My Life', year='1975', duration='262.19057', loudness='-13.386', key='2', tempo='77.464'),
 Row(artist_name='Queens Club', track_name='Lydia', year='0', duration='168.85506', loudness='-3.763', key='2', tempo='140.979'),
 Row(artist_name='Queens Of The Stone Age', track_name='Better Living Through Chemistry', year='2000', duration='349.20444', loudness='-7.478', key='7', tempo='111.433'),
 Row(artist_name='Queen', track_name='Dear Friends (1993 Digital Remaster)', year=

In [12]:
def create_row(song):
    client  = bigtable.Client(project=project_id, admin=True).instance(instance_id)
    table = client.table(table_id)
    row_key=song["artist_name"].encode() + b"\xff\x00" \
            + song["year"].encode() + b"\xff\x00" \
            + song["track_name"].encode() + b"\xff\00"
    row=table.row(row_key)
    row.set_cell(column_family_id,
                "song".encode(),
                json.dumps({"artist_name": song["artist_name"],
                            "track_name": song["track_name"],
                            "year": song["year"]
                            }).encode())
    table.mutate_rows([row])
    return True

songs_indexed=all_queen_songs.map(lambda song: create_row(song)).count()
songs_indexed

457

In [13]:
artist="Queen"
scanned_rows = table.read_rows(start_key=artist.encode()+b'\xff\x00', end_key=artist.encode()+b'\xff\x00\xff')

for row in scanned_rows:
    cell = json.loads(row.cells[column_family_id]["song".encode()][0].value.decode('utf-8'))
    print(cell)

{'artist_name': 'Queen', 'track_name': "'39 (Live) (1994 Digital Remaster)", 'year': '0'}
{'artist_name': 'Queen', 'track_name': 'A Human Body (2009 Digital Remaster)', 'year': '0'}
{'artist_name': 'Queen', 'track_name': 'Another One Bites The Dust (Live)', 'year': '0'}
{'artist_name': 'Queen', 'track_name': 'Bicycle Race (2008 Digital Remaster)', 'year': '0'}
{'artist_name': 'Queen', 'track_name': 'Big Spender (Live)', 'year': '0'}
{'artist_name': 'Queen', 'track_name': 'Bohemian Rhapsody (2008 Digital Remaster)', 'year': '0'}
{'artist_name': 'Queen', 'track_name': 'Bohemian Rhapsody (Live In Frankfurt Bootleg)', 'year': '0'}
{'artist_name': 'Queen', 'track_name': 'Crazy Little Thing Called Love (Live In Montreal)', 'year': '0'}
{'artist_name': 'Queen', 'track_name': 'Death On Two Legs (Dedicated To....) (Live) (1994 Digital Remaster)', 'year': '0'}
{'artist_name': 'Queen', 'track_name': 'Fat Bottomed Girls (2008 Digital Remaster)', 'year': '0'}
{'artist_name': 'Queen', 'track_name': 

### Questions to the solution

- What is the significance of the seperator b"\xff\x00"?
- Why do we need to include the track name in the rowkey?
- What if we wanted the order to be reversed, i.e. the newest songs first?
- Creating the client and the table for every song when writing to the database is an expensive operation. How can it be avoided?
- What if we wanted to retrieve only the Queen-songs from the 1980's. Is that possible with our rowkey?
- What if we wanted to retrieve only the Queen-songs starting with "B". Is that possible with our rowkey?